In [42]:
# !pip install pandas-datareader
# !pip install fix_yahoo_finance
import pandas as pd
import datetime
from datetime import timedelta
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
class EventStudy():
    def __init__(self,file_path):
        try:
            pd.read_csv(file_path)
        except TypeError:
            return f'Wrong File Type.'
        self.dataframe = pd.DataFrame(pd.read_csv(file_path))
        self.company = self.dataframe['Ticker']
        self.market = self.dataframe['Market']
        self.industry = self.dataframe['Industry']
        self.year = self.dataframe['Year']
        self.month = self.dataframe['Month']
        self.day = self.dataframe['Day']
        self.dict = {}
        for i in range(0,len(self.market)):
            self.dict[self.company[i]] = self.market[i]
        eventdate = []
        for i in range(0,len(self.dataframe)):
            eventdate.append(datetime.datetime(self.year[i],self.month[i],self.day[i]))
        self.dataframe['EventDate'] = eventdate
        #suppose there is only one event in a month
        uniquedays = self.dataframe['EventDate'].unique()
        inter_days = []
        for i in range(0,len(uniquedays)-1):
            inter_days.append(int((uniquedays[i+1]-uniquedays[i])/np.timedelta64(1,'D')))
        self.sample_length = min(inter_days)
    
    def reg(self):
        eventdate_set = self.dataframe['EventDate'].unique()
        subdate_cumulative_abnormal_return = pd.DataFrame()
        for date in eventdate_set:
            subevent_df = self.dataframe[self.dataframe.EventDate == date]
            start_date = date - np.timedelta64(self.sample_length,'D')
            end_date = date + np.timedelta64(self.sample_length,'D')
            subevent_company_list = subevent_df['Ticker'].unique()
            subevent_cumulative_abnormal_return = pd.DataFrame()
            for company in subevent_company_list:
                price = web.DataReader([company, self.dict[company]],'yahoo', start_date,end_date)
                close_price = price['Close']
                return_rate = close_price.pct_change()
#                 %matplotlib inline
#                 plt.scatter(return_rate[i],return_rate[m])
                x = return_rate[company]    
                X = sm.add_constant(x)
                y = return_rate[self.dict[company]]
                model = sm.OLS(y,X,missing='drop')
                result = model.fit()
#                 print(result.summary())
                (a,b) = result.params
#                 fig, ax = plt.subplots(figsize=(12,10))
#                 ax.plot(y)
#                 ax.plot(result.fittedvalues)
                return_rate['Abnormal_Return'] = y - x*b - a
                subevent_cumulative_abnormal_return[f'{company}'] = return_rate['Abnormal_Return'].cumsum()[1:]
                subevent_cumulative_abnormal_return['Average_CAR'] = subevent_cumulative_abnormal_return.apply(lambda x: x.sum()/(len(subevent_company_list)),axis = 1)
#                 fig, ax = plt.subplots(figsize=(12,10))
#                 ax.plot(rets['cumu'])
            subdate_cumulative_abnormal_return[f'{date}'] = subevent_cumulative_abnormal_return['Average_CAR']
        Result= subdate_cumulative_abnormal_return.apply(lambda x: x.sum()/len(eventdate_set),axis= 1)
        print(Result)
    
    def visualization(self):
        pass

In [43]:
instance = EventStudy(r'D:data/test.csv')
instance.reg()
# type(timedelta(days = 7))

Date
2015-09-24   -4.252559e-04
2015-09-25   -1.260883e-04
2015-09-28   -1.743744e-03
2015-09-29   -1.777208e-03
2015-09-30   -6.101101e-04
2015-10-01   -5.991538e-04
2015-10-02    2.818851e-04
2015-10-05    1.446139e-03
2015-10-06    1.488816e-03
2015-10-07    1.964716e-03
2015-10-08    2.480509e-03
2015-10-09    2.694299e-03
2015-10-12    2.898047e-03
2015-10-13    2.661583e-03
2015-10-14    2.003073e-03
2015-10-15    2.904174e-03
2015-10-16    3.142853e-03
2015-10-19    3.244044e-03
2015-10-20    2.987634e-03
2015-10-21    2.698129e-03
2015-10-22    3.808994e-03
2015-10-23    4.471718e-03
2015-10-26    4.389116e-03
2015-10-27    4.131106e-03
2015-10-28    5.125619e-03
2015-10-29    5.030604e-03
2015-10-30    4.690685e-03
2015-11-02    5.460131e-03
2015-11-03    5.771155e-03
2015-11-04    5.619204e-03
                  ...     
2016-01-28   -3.250199e-03
2016-01-29   -1.629769e-03
2016-02-01   -1.667852e-03
2016-02-02   -3.009963e-03
2016-02-03   -2.326386e-03
2016-02-04   -1.952067e